# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

* *Тест по логистической регресси показал, что токенизированные библиотекой razdel тексты лучше только на 0,01 в f1 мере для токсичного класса, а в остальном они показали одинаковые результаты, в том числе и средне-взвешанные. А вот KNN показал лучшие результаты f1 меры на дефолтной токенизации. С RandomForest получились крайне низкие значения для токсичного класса, не понял почему. В общем зачёте победителем я бы выбрал встроеный токенизатор, а в конкретном логистическую регрессию на токенизаторе из библиотеки razdel.*

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np

In [2]:
data = pd.read_csv('data/labeled.csv')

In [123]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [124]:
def razdel_tokenizer(text):
    tokens = [token.text for token in razdel_tokenize(text)]
    return tokens

In [125]:
vectorizer_razdel = CountVectorizer(tokenizer=razdel_tokenizer)

X_razdel = vectorizer_razdel.fit_transform(train.comment)
X_razdel_test = vectorizer_razdel.transform(test.comment) 

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [126]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

In [127]:
X.shape, X_test.shape

((12970, 64045), (1442, 64045))

In [128]:
X_razdel.shape, X_razdel_test.shape

((12970, 64980), (1442, 64980))

In [129]:
y = train.toxic.values
y_test = test.toxic.values

In [130]:
clf_razdel = LogisticRegression(C=0.1, class_weight='balanced', max_iter=200)

clf = LogisticRegression(C=0.1, class_weight='balanced')

In [25]:
clf_razdel.fit(X_razdel, y)



LogisticRegression(C=0.1, class_weight='balanced', max_iter=200)

In [26]:
clf.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [27]:
clf.classes_

array([0., 1.])

In [28]:
clf_razdel.classes_

array([0., 1.])

In [29]:
preds = clf.predict(X_test)

preds_razdel = clf_razdel.predict(X_razdel_test)

In [30]:
# Метрики с применения логистической регрессии с применением дефолтного токенизатора
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.84      0.88       969
         1.0       0.73      0.85      0.78       473

    accuracy                           0.85      1442
   macro avg       0.82      0.85      0.83      1442
weighted avg       0.86      0.85      0.85      1442



In [31]:
# Метрики с применения логистической регрессии с применением токенизатора из библиотеки razdel
print(classification_report(y_test, preds_razdel, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.84      0.88       969
         1.0       0.73      0.85      0.79       473

    accuracy                           0.85      1442
   macro avg       0.83      0.85      0.83      1442
weighted avg       0.86      0.85      0.85      1442



In [33]:
# Метрики с применения KNN с применением дефолтного токенизатора
clf2 = KNeighborsClassifier(n_neighbors=10, metric='cosine')
clf2.fit(X, y)
preds2 = clf2.predict(X_test)

print(classification_report(y_test, preds2))

              precision    recall  f1-score   support

         0.0       0.74      0.96      0.84       969
         1.0       0.80      0.32      0.46       473

    accuracy                           0.75      1442
   macro avg       0.77      0.64      0.65      1442
weighted avg       0.76      0.75      0.71      1442



In [34]:
# Метрики с применения KNN с применением токенизатора из библиотеки razdel
clf2_razdel = KNeighborsClassifier(n_neighbors=10, metric='cosine')
clf2_razdel.fit(X_razdel, y)
preds2_razdel = clf2_razdel.predict(X_razdel_test)

print(classification_report(y_test, preds2_razdel))

              precision    recall  f1-score   support

         0.0       0.71      0.95      0.81       969
         1.0       0.67      0.21      0.32       473

    accuracy                           0.71      1442
   macro avg       0.69      0.58      0.57      1442
weighted avg       0.70      0.71      0.65      1442



In [36]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20, )
rf.fit(X, y)

preds_rf = rf.predict(X_test)

print(classification_report(y_test, preds_rf))

              precision    recall  f1-score   support

         0.0       0.67      1.00      0.81       969
         1.0       1.00      0.01      0.03       473

    accuracy                           0.68      1442
   macro avg       0.84      0.51      0.42      1442
weighted avg       0.78      0.68      0.55      1442



In [37]:
rf_razdel = RandomForestClassifier(n_estimators=100, max_depth=20)
rf_razdel.fit(X_razdel, y)

preds_rf_razdel = rf_razdel.predict(X_razdel_test)

print(classification_report(y_test, preds_rf_razdel))


              precision    recall  f1-score   support

         0.0       0.67      1.00      0.80       969
         1.0       1.00      0.01      0.01       473

    accuracy                           0.67      1442
   macro avg       0.84      0.50      0.41      1442
weighted avg       0.78      0.67      0.54      1442



## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [131]:
vectorizer_cv = CountVectorizer(min_df=5, max_df=0.4, max_features=500, ngram_range=(1,2), stop_words=['на', 'по', 'для', 'ко'])
X_cv = vectorizer.fit_transform(train.comment)
X_cv_test = vectorizer.transform(test.comment)

In [132]:
vectorizer_tfidf = TfidfVectorizer(min_df=5, max_df=0.4, max_features=5000, ngram_range=(1,2), stop_words=['на', 'по', 'для', 'ко'])
X_tfidf = vectorizer_tfidf.fit_transform(train.comment)
X_tfidf_test = vectorizer_tfidf.transform(test.comment)

In [133]:
clf_cv = LogisticRegression(C=0.5, class_weight='balanced', max_iter=200)

clf_cv.fit(X_cv, y)

preds_cv = clf_cv.predict(X_cv_test)

print(classification_report(y_test, preds_cv, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.88      0.90       977
         1.0       0.77      0.84      0.80       465

    accuracy                           0.87      1442
   macro avg       0.85      0.86      0.85      1442
weighted avg       0.87      0.87      0.87      1442



In [134]:
proba_preds_test_cv = clf_cv.predict_proba(X_cv_test)[:, 1]

proba_pairs_test_cv = list(zip(train.comment, proba_preds_test_cv))

sorted_proba_pairs_test_cv = sorted(proba_pairs_test_cv, key=lambda x: x[1], reverse=True)

top_toxic_phrases_test_cv = sorted_proba_pairs_test_cv[:10]
for i, (text, proba) in enumerate(top_toxic_phrases_test_cv, start=1):
    print(f"{i}. Вероятность токсичности: {proba:.4f} - {text}")

1. Вероятность токсичности: 1.0000 - Только к чиновникам это не относится. Но, если честно, в последнее время есть улучшения.

2. Вероятность токсичности: 1.0000 - что это было? Кто нибудь объяснит смысл произошедшего? в чем их логика

3. Вероятность токсичности: 1.0000 - в билайне была такая же ситуация лет 7 назад, после этого перестали им пользоваться

4. Вероятность токсичности: 1.0000 - Мне тоже много кому не хочется давать деньги, но приходится. Благо, двощ совсем уж платным не становится.

5. Вероятность токсичности: 1.0000 - э русна на бутылку садис да мимо татарин

6. Вероятность токсичности: 0.9999 - Конечно, я вас не поняла. Мы оба комментируем OMGh5LoL, в той истории нигде не усматривается, что ребёнок испугался. Там скорее родители испугались противодействия родителей остальных детей
7. Вероятность токсичности: 0.9999 - Погодите, ща Ник Кейв придет.

8. Вероятность токсичности: 0.9999 - Отображение времени с высокой точностью... среднесуточный уход-одна секунда. Что-то тут

In [135]:
clf_tfidf = KNeighborsClassifier(n_neighbors=20, metric='cosine', weights='distance', p=1)
clf_tfidf.fit(X_tfidf, y)
preds_tfidf = clf_tfidf.predict(X_tfidf_test)

print(classification_report(y_test, preds_tfidf))

              precision    recall  f1-score   support

         0.0       0.83      0.87      0.85       977
         1.0       0.70      0.63      0.66       465

    accuracy                           0.79      1442
   macro avg       0.77      0.75      0.76      1442
weighted avg       0.79      0.79      0.79      1442



In [136]:
proba_preds_test_tfidf = clf_tfidf.predict_proba(X_tfidf_test)[:, 1]

proba_pairs_test_tfidf = list(zip(train.comment, proba_preds_test_tfidf))

sorted_proba_pairs_test_tfidf = sorted(proba_pairs_test_tfidf, key=lambda x: x[1], reverse=True)

top_toxic_phrases_test_tfidf = sorted_proba_pairs_test_tfidf[:10]
for i, (text, proba) in enumerate(top_toxic_phrases_test_tfidf, start=1):
    print(f"{i}. Вероятность токсичности: {proba:.4f} - {text}")

1. Вероятность токсичности: 1.0000 - которые не могут собраться больше чем в количестве 3 человек так как очко перед товарищем майором жим-жим? Опять у нас далбоёб с РЯЯЯ ПАЧИМУ ИЩО КРЕМЛЬ ШТУРМАМ НИ ВЗЯЛИ ССЫКУНЫ? нарисовался? Что ты лично, охуевший наш, сделал? Хочется сидеть всем на жопе в интернете спокойно, и чтобы кто-то за тебя всё сделал. У нас тут миллионы силовиков с понятным отношением к национализму, а ты тут кукарекаешь про гыгыканье с мемесов . В Российской Империи некоторые меньшинства тупо учили свой родной язык десятилетиями под светом фонаря ночью, потому что выступление было бы самоубийственно и отозвалось бы только смертями лучших людей и ссылками в Сибирь, как было 2 раза с польскими восстаниями. Люди просто ждали своего часа ГОДАМИ, укрепляя базу и готовясь. Жирдяй сделал полезное дело- замутил первый удобоваримый для интеллигентов (а не только для бритоголового пролетариата с германофилией) русский национализм, ставший по-настоящему популярным и обсуждаемым у мол

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [9]:
import nltk
from nltk.corpus import stopwords

In [10]:
nltk.download('stopwords')
stopwords = list(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dmitry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [18]:
vect_cv = CountVectorizer(stop_words=stopwords)

X = vect_cv.fit_transform(train.comment)
X_test = vect_cv.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

clf_cv2 = LogisticRegression(C=0.5, class_weight='balanced', max_iter=20)
clf_cv2.fit(X, y)

preds_cv2 = clf_cv2.predict(X_test)

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
print(classification_report(y_test, preds_cv2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.90      0.89      0.89       958
         1.0       0.79      0.80      0.79       484

    accuracy                           0.86      1442
   macro avg       0.84      0.84      0.84      1442
weighted avg       0.86      0.86      0.86      1442



In [38]:
indexes_cv2 = []

for i in np.sort(clf_cv2.coef_[0])[::-1][:12]:
  indexes_cv2.append(list(clf_cv2.coef_[0]).index(i))
  
vect_cv.get_feature_names_out()[indexes_cv2][:5]

array(['хохлов', 'хохлы', 'дебил', 'тупые', 'сука'], dtype=object)

In [37]:
clf_dt = DecisionTreeClassifier(max_depth=9, class_weight='balanced')
clf_dt.fit(X, y)
preds_dt = clf_dt.predict(X_test)

print(classification_report(y_test, preds_dt))

              precision    recall  f1-score   support

         0.0       0.69      0.99      0.81       958
         1.0       0.84      0.12      0.21       484

    accuracy                           0.70      1442
   macro avg       0.76      0.55      0.51      1442
weighted avg       0.74      0.70      0.61      1442



In [48]:
feature_names_dt = vect_cv.get_feature_names_out()
top_feature_indices_dt = np.argsort(clf_dt.feature_importances_)[::-1][:5]
top_feature_names_dt = [feature_names_dt[i] for i in top_feature_indices_dt]
print(top_feature_names_dt)

['тебе', 'хохлы', 'очень', 'хохлов', 'нахуй']


In [51]:
clf_nb = MultinomialNB(alpha=1.)
clf_nb.fit(X, y)
preds_nb = clf_nb.predict(X_test)

print(classification_report(y_test, preds_nb))

              precision    recall  f1-score   support

         0.0       0.86      0.97      0.91       958
         1.0       0.91      0.69      0.78       484

    accuracy                           0.87      1442
   macro avg       0.89      0.83      0.85      1442
weighted avg       0.88      0.87      0.87      1442



In [61]:
feature_names = vect_cv.get_feature_names_out()
log_prob = clf_nb.feature_log_prob_

top_toxic_words_nb = [feature_names[i] for i in log_prob[1].argsort()[-5:]]
print(top_toxic_words_nb)


['вообще', 'всё', 'тебе', 'просто', 'это']


In [60]:
# Пробовал с разными параметрами, но лучше всего получилось без параметров
rf = RandomForestClassifier()
rf.fit(X, y)

preds_rf = rf.predict(X_test)
print(classification_report(y_test, preds_rf, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.84      0.87      0.86       958
         1.0       0.72      0.68      0.70       484

    accuracy                           0.80      1442
   macro avg       0.78      0.77      0.78      1442
weighted avg       0.80      0.80      0.80      1442



In [62]:
feature_names_rf = vect_cv.get_feature_names_out()
top_feature_indices_rf = np.argsort(rf.feature_importances_)[::-1][:5]
top_feature_names_rf = [feature_names_rf[i] for i in top_feature_indices_rf]
print(top_feature_names_rf)

['хохлы', 'хохлов', 'тебе', 'нахуй', 'очень']
